<a href="https://colab.research.google.com/github/Filarh/Hello-World/blob/master/DeFooocusfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if 'Args' not in locals():
    !apt install -y aria2
    !pip install pygit2==1.12.2
    %cd /content
    !git clone https://github.com/ehristoforu/DeFooocus.git
    %cd /content/DeFooocus
    Args = ""
# Importa la librería de Google Colab
import google.colab
import os
import json

# Activación de Google Drive si es necesario
activar_drive = True #@param {type: "boolean"}

if activar_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    # Rutas deseadas
    nuevo_dir_lora = '/content/drive/MyDrive/MyLoras' #@param {type: "string"}
    nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output' #@param {type: "string"}

    # Archivo launch.py
    launch_py_path = '/content/DeFooocus/launch.py'

    # Verifica si el archivo existe
    if os.path.exists(launch_py_path):
        # Abre el archivo launch.py
        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        # Código a insertar al inicio del launch.py
        codigo_a_insertar = f'''
import os
import json

# Rutas actualizadas
nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

# Archivo de configuración
config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

# Modifica las rutas en el diccionario de configuración
config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

# Guarda los cambios en el archivo de configuración
with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)

'''

        # Escribe el nuevo contenido al inicio del archivo launch.py
        with open(launch_py_path, 'w') as file:
            file.write(codigo_a_insertar + launch_py_content)
    else:
        print("El archivo launch.py no existe en la ubicación proporcionada.")

# Modificación del modelo por defecto en JSON
modificar_modelo = True #@param {type: "boolean"}
nombre_archivo = "art"  #@param {type: "string"}
link_archivo = "https://civitai.com/api/download/models/212479?type=Model&format=SafeTensor&size=full&fp=fp16"  #@param {type: "string"}
aspect_ratio = "1152*896" #@param {type: "string"}
formato = "safetensors"  # @param ["safetensors","ckpt", "bin"]

# Función para modificar el archivo JSON con los parámetros proporcionados
def modificar_json(nombre_archivo: str, link_archivo: str, aspect_ratio: str, formato: str):
    carpeta = '/content/DeFooocus/presets'
    archivos = os.listdir(carpeta)
    json_encontrado = next((archivo for archivo in archivos if archivo.endswith('.json')), None)

    if json_encontrado:
        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        # Modificar los valores necesarios
        archivo_modelo = f"{nombre_archivo}.{formato}"
        data['default_model'] = archivo_modelo
        data['default_aspect_ratio'] = aspect_ratio
        data['default_refiner'] = ""

        # Actualizar el diccionario checkpoint_downloads con el nuevo archivo y link
        data['checkpoint_downloads'] = {archivo_modelo: link_archivo}

        # Establecer el argumento para la segunda celda basado en el nombre del archivo
        global Args
        Args = f"--preset {nombre_archivo}"

        # Simplificar el nombre del archivo de salida
        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, separators=(',', ':'))

        return f"¡Se ha creado el archivo {nuevo_nombre} con los parámetros proporcionados y estructura correcta!"

    else:
        return "No se encontraron archivos JSON en la carpeta especificada."

# Si modificar_modelo es verdadero, modifica el JSON
if modificar_modelo:
    resultado = modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato)
    print(resultado)

# Después de modificar el modelo, descargamos el archivo

# @title 2 - Add the link of your favorite model! Let's use it for training. If it is a private model, we will need your civitAI or huggingface token (you can look at the code, I will not have access to your token)

# Variables para el link y tokens
model_url = link_archivo # Utiliza el link proporcionado en la modificación del modelo

# @markdown - Optional:
tokenCIVITAI = "" # @param {type:"string"}
tokenHUGGINGFACE = "" # @param {type:"string"}

# Función para hacer el download del modelo usando aria2
def download_model_aria2(url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE):
    try:
        # Crear la carpeta de destino si no existe
        model_dir = "/content/DeFooocus/models/checkpoints"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        # Nombre final del archivo
        archivo_modelo = f"{nombre_archivo}.{formato}"
        destino = os.path.join(model_dir, archivo_modelo)

        # Añadir el token al final del enlace si es necesario
        if "huggingface" in url and tokenHUGGINGFACE:
            url += f"&token={tokenHUGGINGFACE}"
        elif "civitai" in url and tokenCIVITAI:
            url += f"&token={tokenCIVITAI}"

        # Comando aria2 para descargar el archivo
        download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}" -o "{archivo_modelo}"'

        # Ejecuta el comando de descarga
        print(f"Descargando desde URL: {download_command}")
        !{download_command}

        # Verifica que el archivo se haya descargado correctamente
        if os.path.exists(destino):
            print(f"El archivo descargado está en: {destino}")
            return destino
        else:
            print("Error al descargar el archivo.")
            return None

    except Exception as e:
        print(f"Error en la descarga: {e}")
        return None



# Descarga el modelo antes de la ejecución final
model_sd = download_model_aria2(model_url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE)
if model_sd:
    print(f"Modelo descargado correctamente: {model_sd}")
else:
    print("Fallo en la descarga del modelo.")

# Ejecución de entry_with_update.py con el argumento opcional para compartir los resultados
print("[DeFooocus] Preparando...")

theme = "dark" #@param ["dark", "light"]
preset = "default" #@param ["default", "realistic", "anime", "lcm", "sai", "turbo", "lighting", "hypersd", "playground_v2.5", "dpo", "spo", "sd1.5"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16" #@param {type: "string"}

if preset != "default":
    args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
    args = f"{advenced_args} --theme {theme}"

# Mezclar args con Args
args = f"{args} {Args}"

print("[DeFooocus] Iniciando...")
!python entry_with_update.py $args


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 2s (747 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123614 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu